In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from deepface import DeepFace
import time
import os
import sys
from tkinter import Tk, filedialog

class FaceEmotionDetection:
    def __init__(self):
        """Initialize the face and emotion detection system"""
        # Initialize face detection
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        self.emotions = ["angry", "disgust", "fear", "happy", "sad", "surprise", "neutral"]

        # Print initialization message
        print("Face & Emotion Detection System Initialized")
        print("Available emotions for detection:", ", ".join(self.emotions))

    def detect_faces_emotions(self, image):
        """
        Detect faces and emotions in an image

        Parameters:
        - image: Input image (numpy array)

        Returns:
        - processed_image: Image with detection results drawn
        - results: List of dictionaries containing face locations and emotions
        """
        # Create a copy of the image to draw on
        processed_image = image.copy()

        # Convert to grayscale for face detection
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Detect faces
        faces = self.face_cascade.detectMultiScale(gray, 1.1, 4)

        # List to store results
        results = []

        # Process each detected face
        for (x, y, w, h) in faces:
            # Extract the face region
            face_img = image[y:y+h, x:x+w]

            try:
                # Analyze face for emotion
                analysis = DeepFace.analyze(
                    face_img,
                    actions=['emotion'],
                    enforce_detection=False,
                    silent=True
                )

                # Get the dominant emotion
                emotion = analysis[0]['dominant_emotion']
                emotion_scores = analysis[0]['emotion']

                # Create result dictionary
                result = {
                    'location': (x, y, w, h),
                    'emotion': emotion,
                    'emotion_scores': emotion_scores
                }

                results.append(result)

                # Draw rectangle around face
                cv2.rectangle(processed_image, (x, y), (x+w, y+h), (0, 255, 0), 2)

                # Display emotion
                label = f"{emotion}"
                cv2.putText(processed_image, label, (x, y-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            except Exception as e:
                print(f"Error processing face: {str(e)}")

        return processed_image, results

    def detect_from_image_path(self, image_path):
        """
        Detect faces and emotions from an image file

        Parameters:
        - image_path: Path to image file
        
        Returns:
        - results: List of dictionaries containing face locations and emotions
        """
        # Read the image
        image = cv2.imread(image_path)

        if image is None:
            print(f"Error: Could not read image from {image_path}")
            return None

        # Process the image
        processed_image, results = self.detect_faces_emotions(image)

        # Display results
        plt.figure(figsize=(12, 8))
        plt.imshow(cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB))
        plt.title("Face & Emotion Detection Results")
        plt.axis('off')
        plt.show()

        # Print detailed results
        print(f"Detected {len(results)} faces:")
        for i, result in enumerate(results):
            print(f"Face {i+1}:")
            print(f"  - Emotion: {result['emotion']}")
            print("  - Emotion scores:")
            for emotion, score in result['emotion_scores'].items():
                print(f"    - {emotion}: {score:.2f}%")
            print()

        return results

    def detect_from_webcam(self, duration=30, display_type='window'):
        """
        Detect faces and emotions from webcam feed

        Parameters:
        - duration: Duration in seconds to run detection
        - display_type: 'window' for OpenCV window or 'matplotlib' for matplotlib display
        """
        # Initialize webcam
        cap = cv2.VideoCapture(0)

        if not cap.isOpened():
            print("Error: Could not open webcam")
            return

        print(f"Running face and emotion detection for {duration} seconds...")
        start_time = time.time()

        if display_type == 'matplotlib':
            plt.figure(figsize=(12, 8))
            
        while time.time() - start_time < duration:
            # Read frame from webcam
            ret, frame = cap.read()

            if not ret:
                print("Error: Could not read frame from webcam")
                break

            # Process the frame
            processed_frame, results = self.detect_faces_emotions(frame)

            if display_type == 'window':
                # Display using OpenCV window (better for VS Code)
                cv2.imshow('Face & Emotion Detection', processed_frame)
            else:
                # Display using matplotlib
                plt.clf()
                plt.imshow(cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB))
                plt.axis('off')
                plt.title(f"Face & Emotion Detection - Time remaining: {int(duration - (time.time() - start_time))}s")
                plt.pause(0.01)

            # Print detection results
            print(f"\rDetected {len(results)} faces. Dominant emotions: " +
                  ", ".join([f"{i+1}: {r['emotion']}" for i, r in enumerate(results)]), end="")

            # Check for quit key
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Release webcam and close windows
        cap.release()
        cv2.destroyAllWindows()
        if display_type == 'matplotlib':
            plt.close()
        print("\nDetection complete")

    def browse_for_image(self):
        """
        Open a file dialog to browse for an image file
        
        Returns:
        - file_path: Path to the selected image file or None if canceled
        """
        # Hide the main tkinter window
        root = Tk()
        root.withdraw()
        
        # Open file dialog
        file_path = filedialog.askopenfilename(
            title="Select an image file",
            filetypes=[
                ("Image files", "*.jpg *.jpeg *.png *.bmp *.gif"),
                ("All files", "*.*")
            ]
        )
        
        # Destroy the tkinter instance
        root.destroy()
        
        return file_path if file_path else None

    def save_processed_image(self, processed_image, original_path=None):
        """
        Save the processed image with detection results
        
        Parameters:
        - processed_image: Image with detection results
        - original_path: Path of the original image to derive save location
        
        Returns:
        - saved_path: Path where the image was saved
        """
        if original_path:
            # Derive output filename from input
            filename, ext = os.path.splitext(original_path)
            output_path = f"{filename}_detected{ext}"
        else:
            # Use default location with timestamp
            timestamp = time.strftime("%Y%m%d-%H%M%S")
            output_path = f"detected_faces_{timestamp}.jpg"
            
        # Save the image
        cv2.imwrite(output_path, processed_image)
        print(f"Processed image saved to: {output_path}")
        return output_path

# Check if running in Jupyter
def is_jupyter():
    try:
        get_ipython()
        return True
    except:
        return False

# Function to demonstrate usage in Jupyter
def run_jupyter_demo():
    detector = FaceEmotionDetection()
    
    print("Face Emotion Detection Tool - Jupyter Mode")
    print("\nChoose an option:")
    print("1. Run webcam detection")
    print("2. Select an image file")
    
    choice = input("Enter your choice (1 or 2): ")
    
    if choice == '1':
        duration = input("Enter duration in seconds (default: 30): ")
        duration = int(duration) if duration.isdigit() else 30
        
        # For Jupyter, use matplotlib display
        detector.detect_from_webcam(duration=duration, display_type='matplotlib')
    elif choice == '2':
        image_path = detector.browse_for_image()
        if image_path:
            print(f"Selected image: {image_path}")
            results = detector.detect_from_image_path(image_path)
            
            save_choice = input("Do you want to save the processed image? (y/n): ")
            if save_choice.lower() == 'y':
                # We need to reprocess the image to save it
                image = cv2.imread(image_path)
                processed_image, _ = detector.detect_faces_emotions(image)
                detector.save_processed_image(processed_image, image_path)
        else:
            print("No image selected")
    else:
        print("Invalid choice")

# Function to run command line interface
def run_command_line():
    import argparse
    
    parser = argparse.ArgumentParser(description='Face and Emotion Detection Tool')
    parser.add_argument('--image', help='Path to the image file')
    parser.add_argument('--webcam', action='store_true', help='Use webcam for detection')
    parser.add_argument('--duration', type=int, default=30, help='Duration for webcam detection in seconds')
    parser.add_argument('--browse', action='store_true', help='Open file browser to select an image')
    parser.add_argument('--save', action='store_true', help='Save the processed image with detection results')
    
    # Ignore unrecognized arguments (helps with Jupyter)
    args, unknown = parser.parse_known_args()
    
    # Initialize the detector
    detector = FaceEmotionDetection()
    
    if args.webcam:
        # Use webcam
        detector.detect_from_webcam(duration=args.duration, display_type='window')
    elif args.image:
        # Use specified image path
        image_path = args.image
        image = cv2.imread(image_path)
        if image is not None:
            processed_image, results = detector.detect_faces_emotions(image)
            
            # Display the image
            cv2.imshow('Face & Emotion Detection Results', processed_image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            
            # Save if requested
            if args.save:
                detector.save_processed_image(processed_image, image_path)
                
            # Print results
            print(f"Detected {len(results)} faces:")
            for i, result in enumerate(results):
                print(f"Face {i+1}:")
                print(f"  - Emotion: {result['emotion']}")
                print("  - Emotion scores:")
                for emotion, score in result['emotion_scores'].items():
                    print(f"    - {emotion}: {score:.2f}%")
                print()
        else:
            print(f"Error: Could not read image from {image_path}")
    elif args.browse:
        # Open file browser to select image
        image_path = detector.browse_for_image()
        if image_path:
            print(f"Selected image: {image_path}")
            image = cv2.imread(image_path)
            if image is not None:
                processed_image, results = detector.detect_faces_emotions(image)
                
                # Display the image
                cv2.imshow('Face & Emotion Detection Results', processed_image)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
                
                # Save if requested
                if args.save:
                    detector.save_processed_image(processed_image, image_path)
                    
                # Print results
                print(f"Detected {len(results)} faces:")
                for i, result in enumerate(results):
                    print(f"Face {i+1}:")
                    print(f"  - Emotion: {result['emotion']}")
                    print("  - Emotion scores:")
                    for emotion, score in result['emotion_scores'].items():
                        print(f"    - {emotion}: {score:.2f}%")
                    print()
            else:
                print(f"Error: Could not read image from {image_path}")
        else:
            print("No image selected")
    else:
        # No arguments provided, show usage instructions
        print("Face & Emotion Detection Tool")
        print("\nUsage options:")
        print("1. Run with webcam: python face_emotion_detector.py --webcam")
        print("2. Analyze image file: python face_emotion_detector.py --image path/to/image.jpg")
        print("3. Browse for image: python face_emotion_detector.py --browse")
        print("4. Save detection results: Add --save to any command")
        print("\nFor more options, use: python face_emotion_detector.py --help")

# Main entry point
if __name__ == "__main__":
    if is_jupyter():
        # Running in Jupyter, use interactive mode
        run_jupyter_demo()
    else:
        # Running as a script, use command line mode
        run_command_line()


Face & Emotion Detection System Initialized
Available emotions for detection: angry, disgust, fear, happy, sad, surprise, neutral
Face Emotion Detection Tool - Jupyter Mode

Choose an option:
1. Run webcam detection
2. Select an image file
Invalid choice


In [ ]:
import cv2
import numpy as np
import pandas as pd
import datetime
import os
import time
from deepface import DeepFace  # For face and emotion detection
import matplotlib.pyplot as plt
from tkinter import Tk, filedialog, simpledialog, messagebox

class AttendanceEmotionSystem:
    def __init__(self, known_faces_dir="known_faces", attendance_file="attendance_log.csv"):
        """
        Initialize the attendance and emotion monitoring system.

        Parameters:
        - known_faces_dir: Directory containing images of known individuals
        - attendance_file: CSV file to store attendance records
        """
        self.known_faces_dir = known_faces_dir
        self.attendance_file = attendance_file
        self.known_face_names = []
        self.emotions = ["angry", "disgust", "fear", "happy", "sad", "surprise", "neutral"]

        # Ensure directories exist
        if not os.path.exists(known_faces_dir):
            os.makedirs(known_faces_dir)
            print(f"Created directory for known faces: {known_faces_dir}")

        # Initialize face detection and recognition
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

        # Initialize attendance log file if it doesn't exist
        if not os.path.exists(attendance_file):
            self._create_attendance_file()

        # Load known faces
        self._load_known_faces()

    def _create_attendance_file(self):
        """Create a new attendance log file with appropriate headers."""
        df = pd.DataFrame(columns=["Name", "Date", "Time", "Emotion"])
        df.to_csv(self.attendance_file, index=False)
        print(f"Created attendance log file: {self.attendance_file}")

    def _load_known_faces(self):
        """Load all known faces from the directory."""
        print("Loading known faces...")
        if not os.path.exists(self.known_faces_dir):
            print(f"Warning: Known faces directory '{self.known_faces_dir}' does not exist.")
            return

        for filename in os.listdir(self.known_faces_dir):
            if filename.endswith((".jpg", ".jpeg", ".png")):
                name = os.path.splitext(filename)[0]
                print(f"Loading face: {name}")
                self.known_face_names.append(name)
                print(f"Successfully loaded face for {name}")

        print(f"Loaded {len(self.known_face_names)} known faces")

    def browse_for_image(self, title="Select an image"):
        """Open a file dialog to browse for an image file."""
        root = Tk()
        root.withdraw()  # Hide the main window
        
        file_path = filedialog.askopenfilename(
            title=title,
            filetypes=[
                ("Image files", "*.jpg *.jpeg *.png *.bmp"),
                ("All files", "*.*")
            ]
        )
        root.destroy()
        return file_path if file_path else None

    def add_new_face_from_file(self):
        """Add a new face from an image file selected via dialog."""
        # Get name for the person
        root = Tk()
        root.withdraw()
        name = simpledialog.askstring("Input", "Enter name for the person:")
        root.destroy()
        
        if not name:
            print("Operation cancelled")
            return
            
        # Select image file
        image_path = self.browse_for_image(f"Select image for {name}")
        
        if not image_path:
            print("No image selected, operation cancelled")
            return
            
        try:
            # Read the image
            frame = cv2.imread(image_path)
            if frame is None:
                print(f"Failed to read image from {image_path}")
                return
                
            # Save the image to known faces directory
            img_path = os.path.join(self.known_faces_dir, f"{name}.jpg")
            cv2.imwrite(img_path, frame)
            print(f"Face saved as {img_path}")
            
            # Add to known faces
            self.known_face_names.append(name)
            print(f"Added {name} to known faces")
            
            # Show confirmation
            root = Tk()
            root.withdraw()
            messagebox.showinfo("Success", f"Successfully added {name} to known faces.")
            root.destroy()
            
        except Exception as e:
            print(f"Error adding face: {str(e)}")
            root = Tk()
            root.withdraw()
            messagebox.showerror("Error", f"Failed to add face: {str(e)}")
            root.destroy()

    def add_new_face_from_webcam(self):
        """Capture and add a new face from webcam."""
        # Get name for the person
        root = Tk()
        root.withdraw()
        name = simpledialog.askstring("Input", "Enter name for the person:")
        root.destroy()
        
        if not name:
            print("Operation cancelled")
            return
            
        try:
            # Initialize webcam
            cap = cv2.VideoCapture(0)
            if not cap.isOpened():
                print("Error: Could not open webcam")
                return
                
            print("Preparing to capture face. Please look at the camera...")
            
            # Create window for preview
            cv2.namedWindow("Capture Face", cv2.WINDOW_NORMAL)
            
            # Countdown display
            for i in range(3, 0, -1):
                ret, frame = cap.read()
                if not ret:
                    print("Failed to capture frame")
                    break
                    
                # Display countdown
                cv2.putText(frame, f"Capturing in {i}...", (30, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)
                cv2.imshow("Capture Face", frame)
                cv2.waitKey(1000)  # Wait for 1 second
            
            # Capture the frame
            ret, frame = cap.read()
            if ret:
                # Display "Captured!" message
                cv2.putText(frame, "Captured!", (30, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)
                cv2.imshow("Capture Face", frame)
                cv2.waitKey(1000)  # Show the captured frame for 1 second
                
                # Save the image
                img_path = os.path.join(self.known_faces_dir, f"{name}.jpg")
                cv2.imwrite(img_path, frame)
                print(f"Face captured and saved as {img_path}")
                
                # Add to known faces
                self.known_face_names.append(name)
                print(f"Added {name} to known faces")
                
                # Show confirmation
                root = Tk()
                root.withdraw()
                messagebox.showinfo("Success", f"Successfully added {name} to known faces.")
                root.destroy()
                
            else:
                print("Failed to capture image")
                
            # Release resources
            cap.release()
            cv2.destroyAllWindows()
            
        except Exception as e:
            print(f"Error adding face: {str(e)}")
            root = Tk()
            root.withdraw()
            messagebox.showerror("Error", f"Failed to add face: {str(e)}")
            root.destroy()

    def mark_attendance(self, name, emotion):
        """
        Mark attendance for a recognized face.

        Parameters:
        - name: Name of the recognized person
        - emotion: Detected emotion
        """
        now = datetime.datetime.now()
        date_str = now.strftime("%Y-%m-%d")
        time_str = now.strftime("%H:%M:%S")

        # Check if attendance already marked for today
        df = pd.read_csv(self.attendance_file)
        today_records = df[(df["Name"] == name) & (df["Date"] == date_str)]

        if today_records.empty:
            new_record = pd.DataFrame({"Name": [name],
                                      "Date": [date_str],
                                      "Time": [time_str],
                                      "Emotion": [emotion]})

            df = pd.concat([df, new_record], ignore_index=True)
            df.to_csv(self.attendance_file, index=False)
            return True, f"Marked attendance for {name}"
        else:
            return False, f"{name} already marked for today"

    def recognize_face(self, frame):
        """
        Recognize a face in the given frame using DeepFace.

        Parameters:
        - frame: Image frame to analyze

        Returns:
        - name: Name of the recognized person or "Unknown"
        - emotion: Detected emotion
        - face_location: Location of the detected face
        """
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, 1.1, 4)

        for (x, y, w, h) in faces:
            face_img = frame[y:y+h, x:x+w]

            try:
                # Analyze face for emotion
                result = DeepFace.analyze(face_img, actions=['emotion'], enforce_detection=False, silent=True)

                emotion = result[0]['dominant_emotion']

                # Try to recognize the face
                name = "Unknown"
                if self.known_face_names:  # Only try recognition if we have known faces
                    try:
                        recognition = DeepFace.find(face_img, self.known_faces_dir, enforce_detection=False, silent=True)
                        if not recognition[0].empty:
                            # Extract the name from the path
                            identity_path = recognition[0].iloc[0]['identity']
                            name = os.path.splitext(os.path.basename(identity_path))[0]
                    except:
                        pass

                return name, emotion, (x, y, w, h)

            except:
                pass

        return None, None, None

    def process_image_file(self):
        """Process a single image file for attendance."""
        # Select image file
        image_path = self.browse_for_image("Select image for attendance")
        
        if not image_path:
            print("No image selected")
            return
            
        try:
            # Read the image
            frame = cv2.imread(image_path)
            if frame is None:
                print(f"Failed to read image from {image_path}")
                return
                
            # Create a copy for displaying results
            display_frame = frame.copy()
            
            # Detect and recognize faces
            marked_attendance = set()
            
            name, emotion, face_location = self.recognize_face(frame)
            
            if name and face_location:
                x, y, w, h = face_location
                
                # Draw rectangle around face
                cv2.rectangle(display_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                
                # Display name and emotion
                label = f"{name} - {emotion}"
                cv2.putText(display_frame, label, (x, y-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                
                # Mark attendance if not already marked
                if name != "Unknown":
                    success, message = self.mark_attendance(name, emotion)
                    if success:
                        marked_attendance.add(name)
                    print(message)
                    
                    # Add status text to image
                    cv2.putText(display_frame, message, (10, 30), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            # Display the processed image
            cv2.imshow("Attendance Processing Result", display_frame)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            
            # Show summary
            if marked_attendance:
                message = f"Marked attendance for: {', '.join(marked_attendance)}"
            else:
                message = "No attendance marked"
                
            root = Tk()
            root.withdraw()
            messagebox.showinfo("Attendance Result", message)
            root.destroy()
            
        except Exception as e:
            print(f"Error processing image: {str(e)}")
            root = Tk()
            root.withdraw()
            messagebox.showerror("Error", f"Failed to process image: {str(e)}")
            root.destroy()

    def run_attendance_system(self, duration=30):
        """
        Run the attendance system for a specified duration using webcam.

        Parameters:
        - duration: Number of seconds to run the system
        """
        # Initialize webcam
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print("Error: Could not open webcam")
            return
            
        start_time = time.time()
        marked_attendance = set()
        
        # Create window for display
        cv2.namedWindow("Attendance System", cv2.WINDOW_NORMAL)
        
        while time.time() - start_time < duration:
            ret, frame = cap.read()
            if not ret:
                print("Failed to capture frame")
                break
                
            # Create a copy for displaying results
            display_frame = frame.copy()
            
            # Display time remaining
            time_remaining = int(duration - (time.time() - start_time))
            cv2.putText(display_frame, f"Time remaining: {time_remaining}s", (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            # Detect and recognize faces
            name, emotion, face_location = self.recognize_face(frame)
            
            if name and face_location:
                x, y, w, h = face_location
                
                # Draw rectangle around face
                cv2.rectangle(display_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                
                # Display name and emotion
                label = f"{name} - {emotion}"
                cv2.putText(display_frame, label, (x, y-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                
                # Mark attendance if not already marked
                if name != "Unknown" and name not in marked_attendance:
                    success, message = self.mark_attendance(name, emotion)
                    if success:
                        marked_attendance.add(name)
                    
                    # Display message on frame
                    cv2.putText(display_frame, message, (10, 60), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    print(message)
            
            # Display the frame
            cv2.imshow("Attendance System", display_frame)
            
            # Check for quit key
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        # Release resources
        cap.release()
        cv2.destroyAllWindows()
        
        # Show summary
        print("\nAttendance Summary:")
        print(f"Total people recognized: {len(marked_attendance)}")
        print(f"People recognized: {', '.join(marked_attendance) if marked_attendance else 'None'}")
        
        if marked_attendance:
            message = f"Marked attendance for: {', '.join(marked_attendance)}"
        else:
            message = "No attendance marked"
            
        root = Tk()
        root.withdraw()
        messagebox.showinfo("Attendance Summary", message)
        root.destroy()

    def generate_reports(self):
        """Generate reports from the attendance data."""
        if not os.path.exists(self.attendance_file):
            print("No attendance data available")
            root = Tk()
            root.withdraw()
            messagebox.showinfo("Report", "No attendance data available")
            root.destroy()
            return

        df = pd.read_csv(self.attendance_file)

        if df.empty:
            print("No attendance records found")
            root = Tk()
            root.withdraw()
            messagebox.showinfo("Report", "No attendance records found")
            root.destroy()
            return

        # Create report text
        report_text = "===== ATTENDANCE REPORT =====\n\n"
        
        # Daily attendance report
        report_text += "=== Daily Attendance Report ===\n"
        daily_count = df.groupby('Date').size()
        report_text += daily_count.to_string() + "\n\n"

        # Person-wise report
        report_text += "=== Person-wise Attendance Report ===\n"
        person_count = df.groupby('Name').size()
        report_text += person_count.to_string() + "\n\n"

        # Emotion analysis
        report_text += "=== Emotion Analysis ===\n"
        emotion_count = df.groupby('Emotion').size()
        report_text += emotion_count.to_string() + "\n"

        # Print to console
        print(report_text)
        
        # Display in message box
        root = Tk()
        root.withdraw()
        messagebox.showinfo("Attendance Report", report_text)
        root.destroy()

        # Visualize emotion distribution
        plt.figure(figsize=(10, 6))
        emotion_count.plot(kind='bar', color='skyblue')
        plt.title('Emotion Distribution')
        plt.xlabel('Emotion')
        plt.ylabel('Count')
        plt.tight_layout()
        plt.show()

def display_menu():
    """Display main menu options."""
    print("\nAttendance and Emotion Recognition System")
    print("=========================================")
    print("1. Add new face from webcam")
    print("2. Add new face from image file")
    print("3. Run attendance system (webcam)")
    print("4. Process image for attendance")
    print("5. Generate attendance reports")
    print("6. Exit")
    choice = input("\nEnter your choice (1-6): ")
    return choice

def main():
    """Main function to run the application."""
    system = AttendanceEmotionSystem()
    
    while True:
        choice = display_menu()
        
        if choice == '1':
            system.add_new_face_from_webcam()
        elif choice == '2':
            system.add_new_face_from_file()
        elif choice == '3':
            duration = input("Enter duration in seconds (default: 30): ")
            duration = int(duration) if duration.strip().isdigit() else 30
            system.run_attendance_system(duration=duration)
        elif choice == '4':
            system.process_image_file()
        elif choice == '5':
            system.generate_reports()
        elif choice == '6':
            print("Exiting application.")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()

Loading known faces...
Loading face: Rahma
Successfully loaded face for Rahma
Loaded 1 known faces

Attendance and Emotion Recognition System
1. Add new face from webcam
2. Add new face from image file
3. Run attendance system (webcam)
4. Process image for attendance
5. Generate attendance reports
6. Exit


In [1]:
!pip install pygame

Defaulting to user installation because normal site-packages is not writeable
